In [29]:
%load_ext autoreload
%autoreload 2

import numpy as np
import random
import networkx as nx
import matplotlib.pyplot as plt
import time
from collections import namedtuple
import pickle

from final_algo import GET_ALGOS, get_algo_lambdas
from paper_sims_util import omega_modularity

AlgoParams = namedtuple('AlgoParams', 'stability_samples M pi')
GraphParams = namedtuple('GraphParams', 'N eta p d ratios')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
num_stocks = 100
run_id = 'first_finance_p_{}'.format(num_stocks)

In [31]:
stocks = np.load('stockdata_data.npy')
stocks_log = np.diff(np.log(stocks), axis = 0)
print(stocks_log.shape)

info = np.load('stockdata_info.npy')
print(info.shape)
print(set(info[:, 1]))

sector = info[:num_stocks, 1]
S = np.cov(stocks_log[:, :num_stocks].T)
X = stocks_log[:, :num_stocks]
print(X.shape)

(1257, 452)
(452, 3)
{'Financials', 'Telecommunications Services', 'Consumer Staples', 'Utilities', 'Energy', 'Materials', 'Industrials', 'Consumer Discretionary', 'Information Technology', 'Health Care'}
(1257, 100)


In [37]:
algo_params = AlgoParams(stability_samples = 10, M = 7./9., pi = 0.8)
graph_params = GraphParams(p=num_stocks, N = 1257, eta=1, ratios=None, d=None)
with open('{}_algo_params.pkl'.format(run_id), 'wb') as f:
    pickle.dump(algo_params, f)
with open('{}_graph_params.pkl'.format(run_id), 'wb') as f:
    pickle.dump(graph_params, f)
with open('{}_sector_X.pkl'.format(run_id), 'wb') as f:
    pickle.dump((sector, X), f)
    
algos = GET_ALGOS(algo_params.stability_samples)
algo_lambdas = get_algo_lambdas(algo_params.M, graph_params.eta, graph_params.N, graph_params.p)
np.random.seed(random.SystemRandom().randint(0, 2**32-2))
ALL_RESULTS = {}
for algo_name, algo in algos.items():
    if algo_name in ['our', 'SH']:
        continue
    print("Currently on {}".format(algo_name))
    lambdas = algo_lambdas[algo_name]
    if algo_name == 'our':
        our_algo_res = algo(X, algo_params.M)
        results = (our_algo_res, None, None)
    else:
        results = algo(X, lambdas, algo_params.pi)
    ALL_RESULTS[algo_name] = results
    with open('{}_{}_algo.pkl'.format(run_id, algo_name), 'wb') as f:
        pickle.dump(results, f)
    omega_hat = results[0]
    mod = omega_modularity(omega_hat, sector)
    print(mod)

Currently on glasso
Working on 0.1
Working on 0.5
Working on 1.0
Working on 2.0
Working on 8.0


/Users/umaroy/Documents/meng/MTP2-algorithm/paper_sims_util.py:34: RuntimeWarning: invalid value encountered in double_scalars
  # print(k_i*k_j / (2*E))


nan
Currently on nbsel
Working on 0.1
Working on 0.5
Working on 1.0
Working on 2.0
Working on 8.0


/Users/umaroy/Documents/meng/MTP2-algorithm/paper_sims_util.py:34: RuntimeWarning: invalid value encountered in double_scalars
  # print(k_i*k_j / (2*E))


nan
Currently on anand
Working on (0.006052784381982069, 1)
Working on (0.06052784381982068, 1)
Working on (0.48422275055856545, 1)
-0.008814141414141318


In [ ]:
mods = {}
for algo_name in algos.keys():
    print(algo_name)
    with open('{}_{}_algo.pkl'.format(run_id, algo_name), 'rb') as f:
        result = pickle.load(f)
    omega_hat = result[0]
    mod = omega_modularity(omega_hat, sector)
    mods[algo_name] = mod
print(mods)

In [34]:
# with open("first_finance_p_100_SH_algo.pkl", 'wb') as f:
#     pickle.dump(ALL_RESULTS['SH'], f)